In [12]:
import os
import pandas as pd
import sqlite3
import yfinance as yf

# Conexão com base local de dados
os.chdir(os.getcwd())
conn = sqlite3.connect(r"C:\databases\assets_portfolio_pessoal.db")
cur = conn.cursor()

In [11]:
ibov_vol = pd.read_sql('SELECT * FROM ibov_volume', conn)
ibov_vol = ibov_vol[(ibov_vol['Date'] >= '2011-01-01') & (ibov_vol['Date'] <= '2020-12-31')]
ibov_vol

,Date,AALR3.SA,ABCB4.SA,ABEV3.SA,AERI3.SA,AESB3.SA,AFLT3.SA,AGRO3.SA,AGXY3.SA,AHEB3.SA,...,VVEO3.SA,WEGE3.SA,WEST3.SA,WHRL3.SA,WHRL4.SA,WIZC3.SA,WLMM3.SA,WLMM4.SA,YDUQ3.SA,ZAMP3.SA
2800,2011-01-03 00:00:00,NaN,296747.0,576145.0,NaN,NaN,0.0,0.0,NaN,0.0,...,NaN,2290288,NaN,0,1200,NaN,NaN,0,825300.0,NaN
2801,2011-01-04 00:00:00,NaN,48294.0,328368.0,NaN,NaN,0.0,61600.0,NaN,0.0,...,NaN,2618824,NaN,0,8000,NaN,NaN,0,663300.0,NaN
2802,2011-01-05 00:00:00,NaN,115014.0,299836.0,NaN,NaN,0.0,0.0,NaN,0.0,...,NaN,3096756,NaN,100,15300,NaN,NaN,0,305100.0,NaN
2803,2011-01-06 00:00:00,NaN,215791.0,731319.0,NaN,NaN,200.0,25100.0,NaN,0.0,...,NaN,2873676,NaN,0,17000,NaN,NaN,0,186300.0,NaN
2804,2011-01-07 00:00:00,NaN,55552.0,1090222.0,NaN,NaN,1200.0,19200.0,NaN,0.0,...,NaN,3000764,NaN,0,11500,NaN,NaN,0,243000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5276,2020-12-22 00:00:00,462500.0,920639.0,15406100.0,296465.0,NaN,500.0,105300.0,NaN,11500.0,...,NaN,5199800,NaN,6200,18000,1739100.0,2300.0,1500,2041500.0,2226400.0
5277,2020-12-23 00:00:00,471500.0,648487.0,11560800.0,534765.0,NaN,600.0,98000.0,NaN,700.0,...,NaN,4806600,NaN,6700,30400,6597300.0,2200.0,2500,2235900.0,1762900.0
5278,2020-12-28 00:00:00,516700.0,750416.0,11958000.0,252210.0,NaN,1800.0,141600.0,NaN,0.0,...,NaN,6838800,NaN,7100,18300,1220300.0,2300.0,2800,1367200.0,1468700.0
5279,2020-12-29 00:00:00,387400.0,624799.0,13638800.0,268260.0,NaN,1900.0,107800.0,NaN,0.0,...,NaN,6130200,NaN,2600,21800,1107300.0,0.0,900,2403200.0,1404400.0


In [16]:
bvsp = yf.download('^BVSP', start='2011-01-01', end='2020-12-31')
bvsp_vol = bvsp['Volume']
bvsp_close = bvsp['Close']
bvsp_returns = bvsp['Close'].pct_change().dropna()

[*********************100%***********************]  1 of 1 completed


C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:689: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [20]:
# BVSP RSI Index
# RSI = 100 - (100 / (1 + RS))
# RS = Average Gain / Average Loss
# Average Gain = Sum of gains over the past 14 periods / 14
# Average Loss = Sum of losses over the past 14 periods / 14

change = bvsp_returns

change_up = change.copy()
change_down = change.copy()

change_up[change_up<0] = 0
change_down[change_down>0] = 0

# Verify that we did not make any mistakes
print(change.equals(change_up+change_down))

# Calculate the rolling average of average up and average down
avg_up = change_up.rolling(14).mean()
avg_down = change_down.rolling(14).mean().abs()

rsi = 100 * avg_up / (avg_up + avg_down)

rsi

True


Date
2011-01-04          NaN
2011-01-05          NaN
2011-01-06          NaN
2011-01-07          NaN
2011-01-10          NaN
                ...    
2020-12-22    70.244874
2020-12-23    71.139128
2020-12-28    72.056895
2020-12-29    73.299583
2020-12-30    72.674238
Name: Close, Length: 2471, dtype: float64

In [21]:
df = pd.DataFrame({'bvsp_vol': bvsp_vol, 'rsi': rsi})
df.to_csv('bvsp_vol_rsi.csv', index=False)